In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, roc_auc_score, f1_score, accuracy_score, precision_score, recall_score
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# llamamos la base preprocesada

import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/proyecto/flights_preprocess.csv')

In [4]:
# debido al tamaño de los datos, con el objetivo de optimizar recursos, vamos a trabajar con una muestra

data = data.sample(frac=0.1, random_state=42)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 557862 entries, 3155306 to 2603752
Data columns (total 54 columns):
 #   Column                                   Non-Null Count   Dtype  
---  ------                                   --------------   -----  
 0   FlightDate                               557862 non-null  object 
 1   Airline                                  557862 non-null  int64  
 2   Origin                                   557862 non-null  int64  
 3   Dest                                     557862 non-null  int64  
 4   CRSDepTime                               557862 non-null  int64  
 5   DepTime                                  557862 non-null  int64  
 6   DepDelayMinutes                          557862 non-null  float64
 7   DepDelay                                 557862 non-null  float64
 8   ArrTime                                  557862 non-null  int64  
 9   ArrDelayMinutes                          557862 non-null  float64
 10  AirTime                       

In [6]:
# definimos nuestras variables explicativas y objetivo (Arr_Delayed - si los vuelos se retrasaron o no)

X = data.drop(columns=['Arr_Delayed', 'FlightDate'])
y = data['Arr_Delayed']

In [7]:
# Dividimos los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# sabemos que existe un desbalanceo en las etiquetas (muy pocos vuelos llegan retrasados). Por ello, vamos a
# balancear generando ejemplos sinteticos de vuelos atrasados por medio de SMOTE

from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


In [9]:
# verificamos

print(pd.Series(y_train).value_counts())

Arr_Delayed
False    362250
True     362250
Name: count, dtype: int64


In [10]:
# vamos a reducir la dimensionalidad de nuestras caracteristicas por medio de componentes principales (PCA)

pca = PCA(n_components= 3)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [11]:
# ahora vamos a entrenar distintos modelos. Nuestro baseline model va a ser una regresión logística, mientras que los demás modelos que entrenaremos serán:
# Random Forest, Gradient Boosting, Support Vector Machine

model_log = LogisticRegression(max_iter=1000).fit(X_train_pca, y_train)
model_rf = RandomForestClassifier(random_state=42).fit(X_train_pca, y_train)


In [12]:
model_gb = GradientBoostingClassifier(random_state=42).fit(X_train_pca, y_train)

In [13]:
# model_svm = SVC(random_state=42).fit(X_train_pca, y_train)

In [14]:
# definimos una funcion para generar las métricas

def metricas(model):
    y_pred = model.predict(X_test_pca)
    metrics = {
        "Model": "{}".format(model),
        "Accuracy": accuracy_score(y_test, y_pred),
        "F1 Score": f1_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred)
    }
    return metrics

In [15]:
# usemos la funcion con los distintos modelos y generemos una lista

resultados = []
metrica_log = metricas(model_log)
metrica_rf = metricas(model_rf)
metrica_gb = metricas(model_gb)
#metrica_svm = metricas(model_svm)
resultados.append(metrica_log)
resultados.append(metrica_rf)
resultados.append(metrica_gb)
# resultados.append(metrica_svm)

In [16]:
# vamos a comparar los modelos según las distintas metricas:

resultados_df = pd.DataFrame(resultados)

# por accuracy

print("Comparación de Modelos: Accuracy")
print(resultados_df.sort_values(by="Accuracy", ascending=False))

Comparación de Modelos: Accuracy
                                         Model  Accuracy  F1 Score  Precision  \
1      RandomForestClassifier(random_state=42)  0.756662  0.396130   0.373940   
0            LogisticRegression(max_iter=1000)  0.521569  0.273979   0.192294   
2  GradientBoostingClassifier(random_state=42)  0.471270  0.301341   0.201011   

     Recall  
1  0.421120  
0  0.476308  
2  0.601627  


In [17]:
# por F1 Score

print("Comparación de Modelos: F1 Score")
print(resultados_df.sort_values(by="F1 Score", ascending=False))

Comparación de Modelos: F1 Score
                                         Model  Accuracy  F1 Score  Precision  \
1      RandomForestClassifier(random_state=42)  0.756662  0.396130   0.373940   
2  GradientBoostingClassifier(random_state=42)  0.471270  0.301341   0.201011   
0            LogisticRegression(max_iter=1000)  0.521569  0.273979   0.192294   

     Recall  
1  0.421120  
2  0.601627  
0  0.476308  


In [18]:
# por Precision

print("Comparación de Modelos: Precision")
print(resultados_df.sort_values(by="Precision", ascending=False))

Comparación de Modelos: Precision
                                         Model  Accuracy  F1 Score  Precision  \
1      RandomForestClassifier(random_state=42)  0.756662  0.396130   0.373940   
2  GradientBoostingClassifier(random_state=42)  0.471270  0.301341   0.201011   
0            LogisticRegression(max_iter=1000)  0.521569  0.273979   0.192294   

     Recall  
1  0.421120  
2  0.601627  
0  0.476308  


In [19]:
# por Recall

print("Comparación de Modelos: Recall")
print(resultados_df.sort_values(by="Recall", ascending=False))

Comparación de Modelos: Recall
                                         Model  Accuracy  F1 Score  Precision  \
2  GradientBoostingClassifier(random_state=42)  0.471270  0.301341   0.201011   
0            LogisticRegression(max_iter=1000)  0.521569  0.273979   0.192294   
1      RandomForestClassifier(random_state=42)  0.756662  0.396130   0.373940   

     Recall  
2  0.601627  
0  0.476308  
1  0.421120  
